# 데이터 로드

In [ ]:
import pandas as pd
df = pd.read_csv(data_path + "5차년도_2차.csv", encoding="cp949")

# Label Encoding
label_map = {
    "fear": 0,
    "surprise": 1,
    "angry": 2,
    "sadness": 3,
    "neutral": 4,
    "happiness": 5,
    "disgust": 6
}
df["y"] = df["상황"].map(label_map)

x_col = '발화문'
y_col = 'y'
input_data = df[[x_col] + [y_col]]
input_data

,발화문,y
0,헐! 나 이벤트에 당첨 됐어.,5
1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,5
2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",5
3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",5
4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,4
...,...,...
19369,나 엘리베이터에 갇혔어.,0
19370,하지만 기분이 나쁜 걸 어떡해?,2
19371,자취방 엘리베이턴데 정전인가봐.,0
19372,나 드디어 프로젝트 끝났어!,5


Split

In [ ]:
from sklearn.model_selection import train_test_split
trval_X, test_X, trval_y, test_y = train_test_split(
    input_data[x_col].tolist(), input_data[y_col].tolist(),
    test_size=0.05, stratify=input_data[y_col], random_state=42)

from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(
    trval_X, trval_y, test_size=0.05, stratify=trval_y, random_state=42)

print(f"            x      y")
print(f"train size: {len(train_X):<5}  {len(train_y):<5}")
print(f"valid size: {len(valid_X):<5}  {len(valid_y):<5}")
print(f"test size : {len(test_X):<5}  {len(test_y):<5}")

            x      y
train size: 17484  17484
valid size: 921    921  
test size : 969    969  


# 토크나이저



In [ ]:
class CustomKoBERTTokenizer:
    def __init__(self, vocab_size=1000):

## vocab 생성

토크나이저 첫 생성 시 vocab 만들기 위해

문장 데이터 입력해서 학습시켜야 함

In [ ]:
corpus = input_data['발화문'].tolist()
print(len(corpus))
corpus[:3]

19374


['헐! 나 이벤트에 당첨 됐어.',
 '내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.',
 '한 명 뽑는 거였는데, 그게 바로 내가 된 거야.']

In [ ]:
custom_tokenizer = CustomKoBERTTokenizer(vocab_size=5000)
custom_tokenizer.train(corpus)

## 구현: 인코드/디코드

토크나이저 encode/decode 기능 구현되어야 함



In [ ]:
test_sentences = [
    "그는 밥을 먹는다",
    "빨리 먹는다",
    "맛있게 먹는다"
]

In [ ]:
print("=== Testing word '먹는다' ===")
for sentence in test_sentences:
    tokens, token_ids = custom_tokenizer.encode(sentence)
    print(f"\nText     : {sentence}")
    print(f"Tokens   : {tokens}")
    print(f"Token ids: {token_ids}")
    print(f"Decoded  : {custom_tokenizer.decode(tokens)}")

=== Testing word '먹는다' ===

Text     : 그는 밥을 먹는다
Tokens   : ['[CLS]', '_그', '는', '_밥', '을', '_먹', '는다', '[SEP]']
Token ids: [2, 61, 19, 695, 107, 432, 3495, 3]
Decoded  : 그는 밥을 먹는다

Text     : 빨리 먹는다
Tokens   : ['[CLS]', '_빨리', '_먹', '는다', '[SEP]']
Token ids: [2, 446, 432, 3495, 3]
Decoded  : 빨리 먹는다

Text     : 맛있게 먹는다
Tokens   : ['[CLS]', '_맛있', '게', '_먹', '는다', '[SEP]']
Token ids: [2, 448, 164, 432, 3495, 3]
Decoded  : 맛있게 먹는다


In [ ]:
# Test with OOV words
oov_sentences = [
    "나는 아이스아메리카노를 마신다",  # "아이스아메리카노" might be OOV
    "그는 스마트폰으로 인터넷서핑을 한다"  # "스마트폰", "인터넷서핑" might be OOV
]

print("\n=== Testing OOV handling ===")
for sentence in oov_sentences:
    tokens, token_ids = custom_tokenizer.encode(sentence)
    print(f"\nText     : {sentence}")
    print(f"Tokens   : {tokens}")
    print(f"Token ids: {token_ids}")
    print(f"Decoded  : {custom_tokenizer.decode(tokens)}")


=== Testing OOV handling ===

Text     : 나는 아이스아메리카노를 마신다
Tokens   : ['[CLS]', '_나', '는', '_아이', '[UNK]', '아', '[UNK]', '리', '카', '노', '를', '_마', '신', '다', '[SEP]']
Token ids: [2, 7, 19, 371, 0, 17, 0, 2365, 3234, 3883, 24, 410, 2764, 229, 3]
Decoded  : 나는 아이아리카노를 마신다

Text     : 그는 스마트폰으로 인터넷서핑을 한다
Tokens   : ['[CLS]', '_그', '는', '_스', '마', '트', '[UNK]', '으로', '_인터넷', '서', '[UNK]', '을', '_한다', '[SEP]']
Token ids: [2, 61, 19, 1951, 2561, 1919, 0, 530, 3842, 117, 0, 107, 1153, 3]
Decoded  : 그는 스마트으로 인터넷서을 한다


## 구현: 모델 입력

In [ ]:
model_path = "drive/MyDrive/2025/KW/Model/"
model_id = "monologg/kobert"

In [ ]:
from transformers import AutoTokenizer
kobert_tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=model_path, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 원래 토크나이저의 vocab_size와 같을 필요는 없음
kobert_tokenizer.vocab_size

8002

In [ ]:
text = "그는 밥을 먹는다"

In [ ]:
inputs = kobert_tokenizer(
            text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=20
        )
inputs

{'input_ids': tensor([[   2, 1191, 2266, 7088, 2010, 5760, 5782,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
inputs = custom_tokenizer(
            text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=20
        )
inputs

{'input_ids': tensor([[   2,   61,   19,  695,  107,  432, 3495,    3,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

## 구현: 스페셜 토큰

In [ ]:
# List of the special token strings
print("All special tokens:    ", kobert_tokenizer.all_special_tokens)

# Corresponding token IDs
print("All special token IDs: ", kobert_tokenizer.all_special_ids)

# Mapping of each special token role
print("Special tokens map:    ", kobert_tokenizer.special_tokens_map)

All special tokens:     ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
All special token IDs:  [0, 3, 1, 2, 4]
Special tokens map:     {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


In [ ]:
for txt in train_X[:5]:
    token_ids = kobert_tokenizer.encode(txt)
    print(txt)
    print(" → tokens[:10]:", token_ids[:10])
    print()

그렇지. 경찰분들 진짜 고생 많으신 것 같애.
 → tokens[:10]: [2, 1205, 54, 975, 6416, 5931, 4368, 993, 6542, 517]

무서워서 잠도 못 잤어.
 → tokens[:10]: [2, 2095, 6553, 7018, 6553, 3945, 5859, 2086, 517, 0]

에이 그 정도는 아니야.
 → tokens[:10]: [2, 3289, 1185, 4099, 5760, 3097, 6844, 54, 3]

아, 진짜? 기대된다. 뭐 찾아올지.
 → tokens[:10]: [2, 3093, 46, 4368, 258, 1269, 54, 2145, 4446, 6973]

날은 아주 좋았어.
 → tokens[:10]: [2, 1407, 7086, 3128, 4208, 6855, 54, 3]



In [ ]:
for txt in train_X[:5]:
    tokens, token_ids = custom_tokenizer.encode(txt)
    print(txt)
    print(" → tokens[:10]:", tokens[:10])
    print(" → ids   [:10]:", token_ids[:10])
    print()

그렇지. 경찰분들 진짜 고생 많으신 것 같애.
 → tokens[:10]: ['[CLS]', '_그렇', '지', '.', '_경찰', '분', '들', '_진짜', '_고생', '_많']
 → ids   [:10]: [2, 275, 34, 13, 1490, 1338, 75, 754, 1160, 286]

무서워서 잠도 못 잤어.
 → tokens[:10]: ['[CLS]', '_무서워서', '_잠', '도', '_못', '_잤', '어', '.', '[SEP]']
 → ids   [:10]: [2, 655, 505, 63, 53, 656, 12, 13, 3]

에이 그 정도는 아니야.
 → tokens[:10]: ['[CLS]', '_에이', '_그', '_정도', '는', '_아니야', '.', '[SEP]']
 → ids   [:10]: [2, 1785, 61, 803, 19, 306, 13, 3]

아, 진짜? 기대된다. 뭐 찾아올지.
 → tokens[:10]: ['[CLS]', '_아', ',', '_진짜', '?', '_기대', '된다', '.', '_뭐', '_찾']
 → ids   [:10]: [2, 300, 40, 754, 112, 801, 2086, 13, 195, 476]

날은 아주 좋았어.
 → tokens[:10]: ['[CLS]', '_날', '은', '_아주', '_좋', '았', '어', '.', '[SEP]']
 → ids   [:10]: [2, 1455, 65, 934, 16, 167, 12, 13, 3]

